# L'importance des modèles

## Données

Repartons des données que nous avons mis en forme, et amenons progressivement une analyse plus avancée

In [1]:
import pandas as pd
df = pd.read_excel("../data/data.xlsx")
df.head()

,Unnamed: 0,id,title,publication_year,abstract
0,0,https://openalex.org/W2159397589,Computational Social Science,2009.0,A field is emerging that leverages the capacit...
1,1,https://openalex.org/W2070907364,Manifesto of computational social science,2012.0,NaN
2,2,https://openalex.org/W3081158114,Computational social science: Obstacles and op...,2020.0,"Data sharing, research ethics, and incentives ..."
3,3,https://openalex.org/W3022499311,Computational Social Science and Sociology,2020.0,The integration of social science with compute...
4,4,https://openalex.org/W3174174150,Integrating explanation and prediction in comp...,2021.0,NaN


## Spacy pour aller vers les modèles

Installer Spacy et un modèle adapté

In [ ]:
#!pip install -U spacy
#!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 792.7 kB/s  0:00:380:00:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [2]:
import spacy
pipeline = spacy.load("en_core_web_md")
doc = pipeline("The language Python rocks, but the day is long.")

In [3]:
type(doc)

spacy.tokens.doc.Doc

Le texte est "structuré" : composé de différents éléments qui n'existaient pas avant

In [4]:
doc[0]

The

POS, lemmatisation, etc.

In [14]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.dep_, token.lemma_, token.is_oov)

The the DET det the False
language language NOUN compound language False
Python Python PROPN compound Python False
rocks rock NOUN ROOT rock False
, , PUNCT punct , False
but but CCONJ cc but False
the the DET det the False
day day NOUN nsubj day False
is be AUX conj be False
long long ADJ acomp long False
. . PUNCT punct . False


Récupérer uniquement les verbes ?

NER

In [15]:
for ent in doc.ents:
    print(ent.text, ent.label_)

the day DATE


In [16]:
result = df["texte"][0:10].apply(pipeline)

In [20]:
for ent in result.iloc[0].ents:
    print(ent.text, ent.label_)

Roger et al. PERSON
2018 DATE
two CARDINAL
one CARDINAL
one CARDINAL


In [5]:
from spacy import displacy
displacy.render(doc, style="ent", 
                jupyter=True)

Plongements

In [11]:
# Compare two documents
doc1 = pipeline(df["texte"][0])
doc2 = pipeline(df["texte"][1000])
print(doc1.similarity(doc2))

0.6429994113259276


## Utiliser des modèles d'HuggingFace : le cas de GliNER

Aller vers les modèles :
- HuggingFace
- Cartes des modèles
- Transformers

Un modèle plus complexe pour l'identification d'entités

Par exemple : https://huggingface.co/urchade/gliner_multi_pii-v1

In [ ]:
#!pip install gliner
from gliner import GLiNER

model = GLiNER.from_pretrained("gliner-community/gliner_large-v2.5")

/opt/anaconda3/envs/pyshs/lib/python3.12/site-packages/huggingface_hub/utils/_validators.py:186: UserWarning: The `resume_download` argument is deprecated and ignored in `snapshot_download`. Downloads always resume whenever possible.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

In [23]:
text = "Il se trouve qu'Émilien aime faire du Python"

labels = ["person"]

entities = model.predict_entities(text, labels)

entities

[{'start': 16,
  'end': 23,
  'text': 'Émilien',
  'label': 'person',
  'score': 0.987253725528717}]

In [ ]:
df["texte"][0:10].apply(lambda x:  model.predict_entities(x, ["mathematics"]))

## Application

Faire de l'analyse de sentiments

Une question : **quelles sont les prises de paroles les plus négatives ?**

- Embarras du choix
    - Par ex : [🚀 distilbert-based Multilingual Sentiment Classification Model
](https://huggingface.co/tabularisai/multilingual-sentiment-analysis)
- Comprendre le modèle / ce qu'il fait
- Importance d'évaluer son résultat